In [1]:
!pip install tensorflow scikit-learn joblib

In [2]:
import pandas as pd
from google.colab import files

uploaded = files.upload()  # chọn file synthetic_landslide_data.csv
df = pd.read_csv("synthetic_landslide_data.csv")


Saving synthetic_landslide_data.csv to synthetic_landslide_data.csv


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Model 1: sinh 3 ngày từ 1 ngày
X_gen = df[[col for col in df.columns if "_input" in col]].values
y_gen = df[[col for col in df.columns if "_day" in col]].values

# Model 2: dự báo FS từ chuỗi 3 ngày
X_fore = y_gen.copy()
y_fore = df[["FS_day1", "FS_day2", "FS_day3"]].values


In [4]:
# Cho Generator
scaler_gen = MinMaxScaler()
X_gen_scaled = scaler_gen.fit_transform(X_gen)
y_gen_scaled = MinMaxScaler().fit_transform(y_gen)

# Cho Forecaster (đầu vào)
scaler_fore = MinMaxScaler()
X_fore_scaled = scaler_fore.fit_transform(X_fore)

# ✅ Cho Forecaster (đầu ra - chính là FS) ← Quan trọng
scaler_fs = MinMaxScaler()
y_fore_scaled = scaler_fs.fit_transform(y_fore)

# Train/test split
Xg_train, Xg_test, yg_train, yg_test = train_test_split(X_gen_scaled, y_gen_scaled, test_size=0.2, random_state=42)
Xf_train, Xf_test, yf_train, yf_test = train_test_split(X_fore_scaled, y_fore_scaled, test_size=0.2, random_state=42)


In [5]:
import tensorflow as tf

model_gen = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(Xg_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(yg_train.shape[1])  # 3 ngày × 9 đặc trưng = 27 hoặc 30
])

model_gen.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_gen.fit(Xg_train, yg_train, validation_data=(Xg_test, yg_test), epochs=50, batch_size=16, verbose=1)

model_gen.save("model_generator.keras")


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.2140 - mae: 0.3721 - val_loss: 0.0831 - val_mae: 0.2385
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0802 - mae: 0.2334 - val_loss: 0.0726 - val_mae: 0.2183
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0703 - mae: 0.2129 - val_loss: 0.0672 - val_mae: 0.2026
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0648 - mae: 0.1975 - val_loss: 0.0649 - val_mae: 0.1947
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0629 - mae: 0.1897 - val_loss: 0.0636 - val_mae: 0.1883
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0613 - mae: 0.1842 - val_loss: 0.0625 - val_mae: 0.1855
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0601 - mae: 0.1814 - val_loss: 0.0620 - val_mae: 0.1842
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0594 - mae: 0.1795 - val_loss: 0.0621 - val_mae: 0.1843
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0595 - mae:

In [6]:
# LSTM input: [samples, timesteps=1, features]
Xf_train_lstm = Xf_train.reshape((-1, 1, Xf_train.shape[1]))
Xf_test_lstm = Xf_test.reshape((-1, 1, Xf_test.shape[1]))

model_fore = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1, Xf_train.shape[1])),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3)  # FS_day1, FS_day2, FS_day3
])

model_fore.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_fore.fit(Xf_train_lstm, yf_train, validation_data=(Xf_test_lstm, yf_test), epochs=50, batch_size=16, verbose=1)

model_fore.save("model_forecaster.keras")


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0385 - mae: 0.1455 - val_loss: 0.0197 - val_mae: 0.1056
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0179 - mae: 0.1024 - val_loss: 0.0144 - val_mae: 0.0867
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0112 - mae: 0.0793 - val_loss: 0.0091 - val_mae: 0.0722
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0074 - mae: 0.0644 - val_loss: 0.0043 - val_mae: 0.0495
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0031 - mae: 0.0428 - val_loss: 0.0014 - val_mae: 0.0265
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.6637e-04 - mae: 0.0213 - val_loss: 7.5176e-04 - val_mae: 0.0196
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.6350e-04 - mae: 0.0155 - val_loss: 6.4291e-04 - val_mae: 0.0170
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.7472e-04 - mae: 0.0137 - val_loss: 4.9773e-04 - val_mae: 0.0152
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/st

In [7]:
import joblib

joblib.dump(scaler_gen, "scaler_generator.save")
joblib.dump(scaler_fore, "scaler_forecaster.save")
joblib.dump(scaler_fs, "scaler_fs_output.save")  # ✅ scaler cho FS output

['scaler_fs_output.save']

In [8]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np

# 🔁 Load models and scalers
model_gen = load_model("model_generator.keras")
model_fore = load_model("model_forecaster.keras")
scaler_gen = joblib.load("scaler_generator.save")
scaler_fore = joblib.load("scaler_forecaster.save")
scaler_fs = joblib.load("scaler_fs_output.save")  # scaler cho FS đầu ra

# 👤 Nhập dữ liệu 1 ngày hiện tại (ví dụ)
# [c, L, gamma, h, u, phi, beta, elevation, slope_type]
sample_input = np.array([[55, 48, 20.3, 12.1, 7.5, 29.2, 22.5, 120, 1]])

# 1️⃣ Dự đoán chuỗi 3 ngày từ 1 ngày
sample_input_scaled = scaler_gen.transform(sample_input)
generated_sequence = model_gen.predict(sample_input_scaled)  # shape: (1, ?)

# ✅ Nếu chưa rõ số đặc trưng, kiểm tra:
print("✅ Shape của chuỗi sinh ra từ model 1:", generated_sequence.shape)
n_features = generated_sequence.shape[1]

# 2️⃣ Dự đoán FS từ chuỗi 3 ngày
sequence_lstm = generated_sequence.reshape((1, 1, n_features))  # (batch, timestep=1, features)
predicted_fs_scaled = model_fore.predict(sequence_lstm)        # (1, 3)
predicted_fs = scaler_fs.inverse_transform(predicted_fs_scaled)

# 3️⃣ In kết quả + gán cảnh báo
for i, fs in enumerate(predicted_fs[0], start=1):
    label = "🟢 An toàn" if fs >= 1.5 else "🟡 Có dấu hiệu" if fs >= 1.0 else "🔴 Nguy cơ cao"
    print(f"📅 FS ngày {i}: {fs:.2f} → {label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
✅ Shape của chuỗi sinh ra từ model 1: (1, 30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
📅 FS ngày 1: 1.98 → 🟢 An toàn
📅 FS ngày 2: 1.78 → 🟢 An toàn
📅 FS ngày 3: 1.68 → 🟢 An toàn


In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 🔁 Load model và scaler
model_gen = load_model("model_generator.keras")
model_fore = load_model("model_forecaster.keras")
scaler_gen = joblib.load("scaler_generator.save")
scaler_fore = joblib.load("scaler_forecaster.save")
scaler_fs = joblib.load("scaler_fs_output.save")

# 🔍 Load lại dữ liệu gốc để kiểm chứng
df = pd.read_csv("synthetic_landslide_data.csv")

# Lấy 100 dòng để test
test_df = df.sample(n=100, random_state=42).reset_index(drop=True)

# Các cột
X_gen_test = test_df[[col for col in df.columns if "_input" in col]].values
y_true_fs = test_df[["FS_day1", "FS_day2", "FS_day3"]].values

# Scale đầu vào cho model 1
X_gen_test_scaled = scaler_gen.transform(X_gen_test)

# ⛓️ Bắt đầu pipeline: Model 1 → Model 2
generated_seq = model_gen.predict(X_gen_test_scaled)  # shape: (100, 27 hoặc 30)
generated_seq_scaled = generated_seq.reshape((generated_seq.shape[0], 1, generated_seq.shape[1]))  # LSTM format

# Dự đoán FS
predicted_fs_scaled = model_fore.predict(generated_seq_scaled)
predicted_fs = scaler_fs.inverse_transform(predicted_fs_scaled)

# So sánh thực tế vs dự đoán
results_df = pd.DataFrame(predicted_fs, columns=["FS_day1_pred", "FS_day2_pred", "FS_day3_pred"])
results_df[["FS_day1_true", "FS_day2_true", "FS_day3_true"]] = y_true_fs

# 🎯 Tính chỉ số MAE / MSE cho từng ngày
for i in range(1, 4):
    mae = mean_absolute_error(results_df[f"FS_day{i}_true"], results_df[f"FS_day{i}_pred"])
    mse = mean_squared_error(results_df[f"FS_day{i}_true"], results_df[f"FS_day{i}_pred"])
    print(f"📅 Ngày {i} → MAE: {mae:.3f}, MSE: {mse:.3f}")

# 🎯 Gán nhãn dự đoán
def classify_fs(fs):
    return "🟢 An toàn" if fs >= 1.5 else "🟡 Có dấu hiệu" if fs >= 1.0 else "🔴 Nguy cơ cao"

for i in range(1, 4):
    results_df[f"Label_day{i}_pred"] = results_df[f"FS_day{i}_pred"].apply(classify_fs)
    results_df[f"Label_day{i}_true"] = results_df[f"FS_day{i}_true"].apply(classify_fs)

# 🔍 Hiển thị 10 dòng mẫu so sánh dự báo vs thực tế
display(results_df.head(10))


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
📅 Ngày 1 → MAE: 0.744, MSE: 0.847
📅 Ngày 2 → MAE: 0.657, MSE: 0.783
📅 Ngày 3 → MAE: 0.697, MSE: 0.781


,FS_day1_pred,FS_day2_pred,FS_day3_pred,FS_day1_true,FS_day2_true,FS_day3_true,Label_day1_pred,Label_day1_true,Label_day2_pred,Label_day2_true,Label_day3_pred,Label_day3_true
0,1.840310,1.552931,1.724391,2.226095,1.443670,1.711332,🟢 An toàn,🟢 An toàn,🟢 An toàn,🟡 Có dấu hiệu,🟢 An toàn,🟢 An toàn
1,1.880694,1.739775,1.807758,1.557464,1.144765,3.864143,🟢 An toàn,🟢 An toàn,🟢 An toàn,🟡 Có dấu hiệu,🟢 An toàn,🟢 An toàn
2,2.020665,1.791591,1.513036,0.972527,1.852930,1.623681,🟢 An toàn,🔴 Nguy cơ cao,🟢 An toàn,🟢 An toàn,🟢 An toàn,🟢 An toàn
3,1.952116,1.339952,1.479210,1.340822,0.756363,1.483652,🟢 An toàn,🟡 Có dấu hiệu,🟡 Có dấu hiệu,🔴 Nguy cơ cao,🟡 Có dấu hiệu,🟡 Có dấu hiệu
4,2.260752,1.659517,1.714001,0.727014,1.199643,1.860013,🟢 An toàn,🔴 Nguy cơ cao,🟢 An toàn,🟡 Có dấu hiệu,🟢 An toàn,🟢 An toàn
5,1.830192,1.728015,1.416054,1.717939,2.903013,1.558942,🟢 An toàn,🟢 An toàn,🟢 An toàn,🟢 An toàn,🟡 Có dấu hiệu,🟢 An toàn
6,1.887201,1.658593,1.502981,1.546285,0.876505,3.086924,🟢 An toàn,🟢 An toàn,🟢 An toàn,🔴 Nguy cơ cao,🟢 An toàn,🟢 An toàn
7,1.918099,1.248715,1.719415,3.801776,1.502265,0.713322,🟢 An toàn,🟢 An toàn,🟡 Có dấu hiệu,🟢 An toàn,🟢 An toàn,🔴 Nguy cơ cao
8,1.615109,1.559373,1.551354,1.147907,1.993844,2.340613,🟢 An toàn,🟡 Có dấu hiệu,🟢 An toàn,🟢 An toàn,🟢 An toàn,🟢 An toàn
9,1.886666,1.399953,1.450052,1.398004,2.441877,1.753383,🟢 An toàn,🟡 Có dấu hiệu,🟡 Có dấu hiệu,🟢 An toàn,🟡 Có dấu hiệu,🟢 An toàn


In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report

# Ẩn cảnh báo TensorFlow không cần thiết
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore", category=UserWarning)
tf.get_logger().setLevel('ERROR')

# 🔁 Load models và scalers
model_gen = load_model("model_generator.keras")
model_fore = load_model("model_forecaster.keras")
scaler_gen = joblib.load("scaler_generator.save")
scaler_fore = joblib.load("scaler_forecaster.save")
scaler_fs = joblib.load("scaler_fs_output.save")

# 🔍 Load dữ liệu test từ file gốc
df = pd.read_csv("synthetic_landslide_data.csv")
test_df = df.sample(n=100, random_state=42).reset_index(drop=True)

# Tách dữ liệu
X_gen_test = test_df[[col for col in df.columns if "_input" in col]].values
y_true_fs = test_df[["FS_day1", "FS_day2", "FS_day3"]].values

# 👉 Scale input cho model 1
X_gen_test_scaled = scaler_gen.transform(X_gen_test)

# 👉 Predict 1 lần duy nhất từ model 1 (MLP)
generated_seq = model_gen.predict(X_gen_test_scaled)

# 👉 Reshape cho LSTM
generated_seq_lstm = generated_seq.reshape((generated_seq.shape[0], 1, generated_seq.shape[1]))

# 👉 Predict 1 lần duy nhất từ model 2 (LSTM)
predicted_fs_scaled = model_fore.predict(generated_seq_lstm)
predicted_fs = scaler_fs.inverse_transform(predicted_fs_scaled)

# 👉 Tạo bảng kết quả
results_df = pd.DataFrame(predicted_fs, columns=["FS_day1_pred", "FS_day2_pred", "FS_day3_pred"])
results_df[["FS_day1_true", "FS_day2_true", "FS_day3_true"]] = y_true_fs

# 🎯 Đánh giá MAE/MSE cho từng ngày
for i in range(1, 4):
    mae = mean_absolute_error(results_df[f"FS_day{i}_true"], results_df[f"FS_day{i}_pred"])
    mse = mean_squared_error(results_df[f"FS_day{i}_true"], results_df[f"FS_day{i}_pred"])
    print(f"📅 Ngày {i} → MAE: {mae:.3f}, MSE: {mse:.3f}")

# 🎯 Hàm gán nhãn cảnh báo
def classify_fs(fs):
    if fs >= 1.5: return "An toàn"
    elif fs >= 1.0: return "Có dấu hiệu"
    else: return "Nguy cơ cao"

# Gán nhãn
for i in range(1, 4):
    results_df[f"Label_day{i}_pred"] = results_df[f"FS_day{i}_pred"].apply(classify_fs)
    results_df[f"Label_day{i}_true"] = results_df[f"FS_day{i}_true"].apply(classify_fs)

# ✅ Tính chính xác dự báo nhãn
for i in range(1, 4):
    y_true = results_df[f"Label_day{i}_true"]
    y_pred = results_df[f"Label_day{i}_pred"]
    acc = np.mean(y_true == y_pred)
    print(f"🎯 Độ chính xác phân loại cảnh báo – Ngày {i}: {acc*100:.2f}%")
    print(classification_report(y_true, y_pred, digits=3))

# ✅ Hiển thị vài dòng kết quả mẫu
display(results_df.head(10))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
📅 Ngày 1 → MAE: 0.744, MSE: 0.847
📅 Ngày 2 → MAE: 0.657, MSE: 0.783
📅 Ngày 3 → MAE: 0.697, MSE: 0.781
🎯 Độ chính xác phân loại cảnh báo – Ngày 1: 53.00%
              precision    recall  f1-score   support

     An toàn      0.530     1.000     0.693        53
 Có dấu hiệu      0.000     0.000     0.000        32
 Nguy cơ cao      0.000     0.000     0.000        15

    accuracy                          0.530       100
   macro avg      0.177     0.333     0.231       100
weighted avg      0.281     0.530     0.367       100

🎯 Độ chính xác phân loại cảnh báo – Ngày 2: 56.00%
              precision    recall  f1-score   support

     An toàn      0.589     0.914     0.716        58
 Có dấu hiệu      0.300     0.115     0.167        26
 Nguy cơ cao      0.000     0.000     0.000        16

    accuracy                          0.560       100
   macro avg      0.296     0.343     0.294       100
weighted avg 

,FS_day1_pred,FS_day2_pred,FS_day3_pred,FS_day1_true,FS_day2_true,FS_day3_true,Label_day1_pred,Label_day1_true,Label_day2_pred,Label_day2_true,Label_day3_pred,Label_day3_true
0,1.840310,1.552931,1.724391,2.226095,1.443670,1.711332,An toàn,An toàn,An toàn,Có dấu hiệu,An toàn,An toàn
1,1.880694,1.739775,1.807758,1.557464,1.144765,3.864143,An toàn,An toàn,An toàn,Có dấu hiệu,An toàn,An toàn
2,2.020665,1.791591,1.513036,0.972527,1.852930,1.623681,An toàn,Nguy cơ cao,An toàn,An toàn,An toàn,An toàn
3,1.952116,1.339952,1.479210,1.340822,0.756363,1.483652,An toàn,Có dấu hiệu,Có dấu hiệu,Nguy cơ cao,Có dấu hiệu,Có dấu hiệu
4,2.260752,1.659517,1.714001,0.727014,1.199643,1.860013,An toàn,Nguy cơ cao,An toàn,Có dấu hiệu,An toàn,An toàn
5,1.830192,1.728015,1.416054,1.717939,2.903013,1.558942,An toàn,An toàn,An toàn,An toàn,Có dấu hiệu,An toàn
6,1.887201,1.658593,1.502981,1.546285,0.876505,3.086924,An toàn,An toàn,An toàn,Nguy cơ cao,An toàn,An toàn
7,1.918099,1.248715,1.719415,3.801776,1.502265,0.713322,An toàn,An toàn,Có dấu hiệu,An toàn,An toàn,Nguy cơ cao
8,1.615109,1.559373,1.551354,1.147907,1.993844,2.340613,An toàn,Có dấu hiệu,An toàn,An toàn,An toàn,An toàn
9,1.886666,1.399953,1.450052,1.398004,2.441877,1.753383,An toàn,Có dấu hiệu,Có dấu hiệu,An toàn,Có dấu hiệu,An toàn


In [11]:
# Sau khi huấn luyện xong cả 2 mô hình:
model_gen.save("model_generator.keras")
model_fore.save("model_forecaster.keras")

import joblib
joblib.dump(scaler_gen, "scaler_generator.save")
joblib.dump(scaler_fore, "scaler_forecaster.save")
joblib.dump(scaler_fs, "scaler_fs_output.save")


['scaler_fs_output.save']

In [14]:
from google.colab import files
files.download("model_generator.keras")
files.download("model_forecaster.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
from google.colab import files

files.download("scaler_generator.save")
files.download("scaler_forecaster.save")
files.download("scaler_fs_output.save")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>